In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
import pandas as pd
import requests
import re
import time
import numpy as np
import holidays
from datetime import datetime, timedelta
import time
from scipy.stats import skew, kurtosis
import ast
from polygon import RESTClient
from polygon.rest.models.request import RequestOptionBuilder
api_key = "HsOX1rl8t3B1AJfl4O2uqscUp372l7EF"
ticker = "SPY240802C00530000"
from scipy.stats import kurtosis, skew
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
from requests.exceptions import HTTPError

In [2]:
date = "2024-07-29"
type ="P"
strike = 545

In [3]:
# Define US holidays for the year 2024
us_holidays = holidays.US(years=2024)

# Function to calculate the next working day
def next_working_day(date):
    next_day = date + timedelta(days=1)
    
    # Check if the next day is a weekend or holiday and adjust accordingly
    while next_day.weekday() in (5, 6) or next_day in us_holidays:
        next_day += timedelta(days=1)
    
    return next_day
next_work = datetime.strptime(date, '%Y-%m-%d')
next_working_date = next_working_day(next_work)

In [4]:
# Function to calculate the next working day
def next_working_day(date):
    next_day = date + timedelta(days=1)
    
    # Check if the next day is a weekend or holiday and adjust accordingly
    while next_day.weekday() in (5, 6) or next_day in us_holidays:
        next_day += timedelta(days=1)
    
    return next_day

# Calculate the next working day
next_work = datetime.strptime(date, '%Y-%m-%d')
next_working_date = next_working_day(next_work)
expiration_date = next_working_date.strftime("%y%m%d")

# Print the result
print(f"Trading Date: {date}")
print(f"Expiration Date: {next_working_date.strftime('%Y-%m-%d')}")


# Function to replace the strike value in the given string
def replace_strike(symbol, strike, expiration_date):
    # Format the strike value to be 8 digits long, padded with zeros
    formatted_strike = f"{strike * 1000:08d}"  # Multiply by 1000 to get the correct format
    # Construct the new symbol
    new_symbol = f"SPY{expiration_date}{type}{formatted_strike}"
    return new_symbol
expiration_date = next_working_date.strftime("%y%m%d")
# Replace the date and strike in the ticker
ticker = replace_strike(ticker, strike, expiration_date)
print(f"Ticker: {ticker}")

Trading Date: 2024-07-29
Expiration Date: 2024-07-30
Ticker: SPY240730P00545000


In [5]:
def get_option_price(ticker, date):
    # Convert the date to the correct format (YYYY-MM-DD)
    API_KEY = 'HsOX1rl8t3B1AJfl4O2uqscUp372l7EF'
    url = f"https://api.polygon.io/v1/open-close/O:{ticker}/{date}?adjusted=true&apiKey={API_KEY}"
    print(url)  # For debugging purposes
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            if 'close' in data:
                return data['close']
            else:
                raise ValueError("Response JSON does not contain 'close' key")
        except ValueError as e:
            print(f"Error parsing JSON: {e}")
    else:
        print(f"Request failed with status code {response.status_code}")
    return None

In [6]:
open_trade_price = get_option_price(ticker, date) *100
close_trade_date = next_working_date.date() 
close_trade_price = get_option_price(ticker, close_trade_date) *100
beneficio = close_trade_price - open_trade_price
perc = beneficio*100/open_trade_price
beneficio = round(beneficio,0)*100
perc = round(perc,0) 
print(f"Precio de apertura: {open_trade_price}")
print(f"Precio de cierre  : {close_trade_price}")
print(f"Beneficio: {beneficio}$- {perc}%")

https://api.polygon.io/v1/open-close/O:SPY240730P00545000/2024-07-29?adjusted=true&apiKey=HsOX1rl8t3B1AJfl4O2uqscUp372l7EF
https://api.polygon.io/v1/open-close/O:SPY240730P00545000/2024-07-30?adjusted=true&apiKey=HsOX1rl8t3B1AJfl4O2uqscUp372l7EF
Precio de apertura: 217.0
Precio de cierre  : 595.0
Beneficio: 37800.0$- 174.0%
